In [1]:
import pandas as pd
from tqdm import tqdm
import numpy as np
import random
import os
import time#可以用来简单地记录时间
import matplotlib.pyplot as plt#画图
from sklearn.metrics import roc_auc_score, precision_recall_curve, roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.metrics import  average_precision_score

import torch#深度学习的pytoch平台
from torch import Tensor
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.utils.data import TensorDataset, Dataset, DataLoader, RandomSampler, SequentialSampler
from torch.nn.utils.rnn import pad_sequence

In [2]:
# from models.lstm import LSTMModel
# from trainer.train_and_evaluate import ModelTrainer, ModelEvaluator
# from data_provider.dataset_generate import TimeSeriesDataset
from models.lstm_itransformer import lstm_itransformerModel

## 加载数据

In [3]:
# database_names_all = ['ams', 'eicu', 'inspire', 'mimiciii', 'mimiciv', 'salz', 'zhejiang']
database_names_all = ['ams', 'zhejiang', 'inspire', 'salz','eicu', 'mimic']
# database_names_all = ['mimiciv']

selected_database_num = -1
internal_database = database_names_all[selected_database_num]  # ''
external_database = database_names_all.copy()
external_database.remove(internal_database) # []
print(internal_database, external_database)

mimic ['ams', 'zhejiang', 'inspire', 'salz', 'eicu']


In [4]:
file_path = "E:\\Research\\Time series research\\Federated learning Time series research\\0.data\\Multi-center time series data\\"
internal_data_path = 'icu_mortality_' + internal_database + '.csv'

In [5]:
df_internal = pd.read_csv(file_path + internal_data_path)
df_internal

,id,gender,age,height,weight,bmi,admission_type,death_hosp,los_icu_day,hr,...,alkaline_phosphatase,lymphocytes,bicarbonate,urineoutput,ventilation,norepinephrine,epinephrine,dobutamine,dopamine,hr_encode
0,5,0,1.017520,-0.021828,-0.793136,-0.999870,1,0,1.73,1,...,-0.463749,-1.139631,-0.161607,-0.036645,1,0,0,0,0,0.001389
1,5,0,1.017520,-0.021828,-0.793136,-0.999870,1,0,1.73,2,...,-0.463749,-1.139631,-0.161607,-0.380820,1,0,0,0,0,0.002778
2,5,0,1.017520,-0.021828,-0.793136,-0.999870,1,0,1.73,3,...,-0.463749,-1.139631,-0.161607,-0.380820,1,0,0,0,0,0.004167
3,5,0,1.017520,-0.021828,-0.793136,-0.999870,1,0,1.73,4,...,-0.463749,-1.139631,-0.161607,-0.208732,1,0,0,0,0,0.005556
4,5,0,1.017520,-0.021828,-0.793136,-0.999870,1,0,1.73,5,...,-0.463749,-1.139631,-0.161607,-0.251754,1,0,0,0,0,0.006944
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3918410,76271,1,-0.637632,0.583969,-0.542062,-0.932103,0,0,1.17,25,...,-0.106978,0.270036,0.036672,2.200495,0,0,0,0,0,0.034722
3918411,76271,1,-0.637632,0.583969,-0.542062,-0.932103,0,0,1.17,26,...,-0.106978,0.270036,0.036672,1.985385,0,0,0,0,0,0.036111
3918412,76271,1,-0.637632,0.583969,-0.542062,-0.932103,0,0,1.17,27,...,-0.106978,0.270036,0.036672,0.608684,0,0,0,0,0,0.037500
3918413,76271,1,-0.637632,0.583969,-0.542062,-0.932103,0,0,1.17,28,...,-0.106978,0.270036,0.036672,-0.595929,0,0,0,0,0,0.038889


In [6]:
print(df_internal.groupby('id')['death_hosp'].last().value_counts())

death_hosp
0    35436
1     5614
Name: count, dtype: int64


In [7]:
train_ids, test_ids = train_test_split(
    df_internal['id'].unique(),  # 按患者ID划分
    test_size=0.2,      # 测试集比例
    random_state=42,    # 随机种子
    stratify=df_internal.groupby('id')['death_hosp'].last()  # 按患者最终标签分层
)
train_ids

array([40804, 52798, 37874, ..., 24565, 47758, 43247], dtype=int64)

In [33]:
len(train_ids), len(test_ids)

(32840, 8210)

In [8]:
# 获取划分后的DataFrame
train_df = df_internal[df_internal['id'].isin(train_ids)]
test_df = df_internal[df_internal['id'].isin(test_ids)]

print("训练集比例:")
print(train_df.groupby('id')['death_hosp'].last().value_counts(normalize=True))
print("\n测试集比例:")
print(test_df.groupby('id')['death_hosp'].last().value_counts(normalize=True))

训练集比例:
death_hosp
0    0.863246
1    0.136754
Name: proportion, dtype: float64

测试集比例:
death_hosp
0    0.863216
1    0.136784
Name: proportion, dtype: float64


In [34]:
print("训练集:")
print(train_df.groupby('id')['death_hosp'].last().value_counts())
print("\n测试集:")
print(test_df.groupby('id')['death_hosp'].last().value_counts())

训练集:
death_hosp
0    28349
1     4491
Name: count, dtype: int64

测试集:
death_hosp
0    7087
1    1123
Name: count, dtype: int64


## 数据生成器

In [9]:
class TimeSeriesDataset(Dataset):
    def __init__(self, df, feature_cols, window_size=24, forecast_horizon=24, stride=1, 
                 mode='sliding', shuffle=True, label=['death_hosp'], random_state=42, max_len=None):
        """
        初始化时间序列数据集
        
        参数:
            df: 包含所有数据的DataFrame
            feature_cols: 使用的特征列名列表
            window_size: 输入序列长度
            forecast_horizon: 预测时间范围
            mode: 'sliding'滑动窗口，'cumulative'累积窗口，‘fix’固定时间窗口
            shuffle: 是否打乱数据顺序
            random_state: 随机种子
            label: 标签
        """
        self.df = df
        self.feature_cols = feature_cols
        self.window_size = window_size
        self.forecast_horizon = forecast_horizon
        self.mode = mode
        self.shuffle = shuffle
        self.random_state = random_state
        self.max_len = max_len
        self.indices = []
        self.stride = stride
        self.label = label
        

        # if len(label)==1:
        #     self.label = label[0]
        # else:
        #     self.label = label
        
        # 预计算所有可能的序列索引
        self._precompute_indices()
        
    def _precompute_indices(self):
        """计算所有有效的序列索引"""
        random.seed(self.random_state)
        
        for pid, group in tqdm(self.df.groupby('id')):
            group = group.sort_values('hr')
            max_hr = group['hr'].max()
            
            if self.mode == 'sliding':
                for start in range(1, max_hr - self.window_size, self.stride):
                    end = start + self.window_size
                    forecast_end = end + self.forecast_horizon
                    if len(group[(group['hr'] >= start) & (group['hr'] < end)]) == self.window_size:
                        y = []
                        for label in self.label:
                            condition = (group['hr'] >= end) & (group['hr'] < forecast_end) & (group[label] == 1)
                            y.append(int(condition.any()))
                        self.indices.append((pid, start, end, y))
                        
            elif self.mode == 'cumulative':
                # for end in range(max_hr, max_hr+1, self.stride):
                # for end in range(13, 14, self.stride):
                for end in range(1, max_hr + 1, self.stride):
                    start = 1
                    forecast_end = end + self.forecast_horizon
                    y = []
                    for label in self.label:
                        condition = (group['hr'] >= end) & (group['hr'] <= forecast_end) & (group[label] == 1)
                        y.append(int(condition.any()))
                    self.indices.append((pid, start, end, y))
                    
            elif self.mode == 'fix':
                start = 1
                end = start + self.window_size
                if len(group[(group['hr'] >= start) & (group['hr'] < end)]) == self.window_size:
                    y = []
                    for label in self.label:
                        condition = (group['hr'] >= end) & (group[label] == 1)
                        y.append(int(condition.any()))
                    self.indices.append((pid, start, end, y))
                        
                    # # condition = (group['hr'] >= end) & (group['death_hosp'] == 1)
                    # condition = group['death_hosp'] == 1
                    # y = int(condition.any())
                    # self.indices.append((pid, start, end, y))
                
        
        if self.shuffle:
            random.shuffle(self.indices)
    
    def __len__(self):
        return len(self.indices)
    
    def __getitem__(self, idx):
        pid, start, end, y = self.indices[idx]
        group = self.df[self.df['id'] == pid].sort_values('hr')
        
        # 获取特征序列
        X = group[(group['hr'] >= start) & (group['hr'] <= end)][self.feature_cols].values
        
        # 转换为torch张量
        X_tensor = torch.FloatTensor(X)
        y_tensor = torch.FloatTensor(y)

        if self.max_len is not None:
            X_tensor = X_tensor[-self.max_len:]  # 截断到最大长度
            
        seq_len = len(X_tensor)
        
        return X_tensor, y_tensor, seq_len

In [10]:
mask_cols = ['heart_rate', 'sbp', 'mbp', 'dbp', 'resp_rate', 'temperature', 'spo2', 'albumin', 'aniongap', 'bun', 'calcium', 'chloride', 
                'creatinine', 'glucose', 'sodium', 'potassium', 'fibrinogen', 'inr', 'pt', 'ptt', 'hematocrit', 'hemoglobin', 'platelet', 'wbc', 
                'alt', 'ast', 'bilirubin', 'pao2', 'paco2', 'fio2', 'pao2fio2ratio', 'ph', 'baseexcess', 'lactate', 'sao2', 'troponin', 'magnesium', 
                'bnp', 'neutrophils', 'gcs', 'alkaline_phosphatase', 'norepinephrine', 'epinephrine', 'dobutamine', 'dopamine', 'ventilation',
                'lymphocytes', 'bicarbonate', 'urineoutput',]

mask_cols = [i+'_mask' for i in mask_cols]
# mask_cols = mask_cols + ['hr_encode']

In [11]:
# 定义特征列
feature_cols = ['gender', 'age', 'height', 'weight',
                'heart_rate', 'sbp', 'mbp', 'resp_rate', 'temperature', 'spo2', 'albumin', 'aniongap', 'bun', 'calcium', 'chloride', 
                'creatinine', 'glucose', 'sodium', 'potassium', 'fibrinogen', 'inr', 'pt', 'ptt', 'hematocrit', 'hemoglobin', 'platelet', 'wbc', 
                'alt', 'ast', 'bilirubin', 'pao2', 'paco2', 'fio2', 'pao2fio2ratio', 'ph', 'baseexcess', 'lactate', 'sao2', 'troponin', 'magnesium', 
                'bnp', 'neutrophils', 'gcs', 'alkaline_phosphatase', 'norepinephrine', 'epinephrine', 'dobutamine', 'dopamine', 'ventilation',
                'lymphocytes', 'bicarbonate', 'urineoutput', #'hr_encode',
               ]
# # 定义特征列
# feature_cols = ['gender', 'ventilation', 'norepinephrine', 'epinephrine', 'dobutamine', 'dopamine', 'admission_type',
#                 'age', 'height', 'weight', 'bmi',
#                 'heart_rate', 'sbp', 'mbp', 'dbp', 'resp_rate', 'temperature', 'spo2', 'albumin', 'aniongap', 'bun', 'calcium', 'chloride', 
#                 'creatinine', 'glucose', 'sodium', 'potassium', 'fibrinogen', 'inr', 'pt', 'ptt', 'hematocrit', 'hemoglobin', 'platelet', 'wbc', 
#                 'alt', 'ast', 'bilirubin', 'pao2', 'paco2', 'fio2', 'pao2fio2ratio', 'ph', 'baseexcess', 'lactate', 'sao2', 'troponin', 'magnesium', 
#                 'bnp', 'neutrophils', 'gcs', 'alkaline_phosphatase', 'norepinephrine', 'epinephrine', 'dobutamine', 'dopamine', 
#                 'lymphocytes', 'bicarbonate', 'urineoutput',
#                ]
# feature_cols = feature_cols + mask_cols    

# 创建数据集和数据加载器
# train_dataset = TimeSeriesDataset(train_df, feature_cols, window_size=24, forecast_horizon=24, mode='sliding', shuffle=True)
# test_dataset = TimeSeriesDataset(test_df, feature_cols, window_size=24, forecast_horizon=24, mode='sliding', shuffle=False)

# train_dataset = TimeSeriesDataset(train_df, feature_cols, window_size=24, mode='fix', shuffle=False)
# test_dataset = TimeSeriesDataset(test_df, feature_cols, window_size=24, mode='fix', shuffle=False)

train_dataset = TimeSeriesDataset(train_df, feature_cols, mode='cumulative', shuffle=False, stride=4, label=['death_hosp'])
test_dataset = TimeSeriesDataset(test_df, feature_cols, mode='cumulative', shuffle=False, stride=4, label=['death_hosp'])

100%|█████████████████████████████████████████████████████████████████████████████| 8210/8210 [00:16<00:00, 486.25it/s]


In [12]:
# 提取标签列
ys = np.array([y[0] for (pid, start, end, y) in train_dataset.indices])
unique, counts = np.unique(ys, return_counts=True)
label_dist = dict(zip(unique, counts))

print("\n使用numpy统计:")
for label, count in label_dist.items():
    print(f"{label}: {count} 个 ({(count/len(ys))*100:.1f}%)")


使用numpy统计:
0: 764577 个 (96.5%)
1: 27870 个 (3.5%)


In [13]:
# 提取标签列
ys = np.array([y[0] for (pid, start, end, y) in test_dataset.indices])
unique, counts = np.unique(ys, return_counts=True)
label_dist = dict(zip(unique, counts))

print("\n使用numpy统计:")
for label, count in label_dist.items():
    print(f"{label}: {count} 个 ({(count/len(ys))*100:.1f}%)")


使用numpy统计:
0: 194489 个 (96.5%)
1: 6973 个 (3.5%)


In [14]:
train_dataset.indices[:100]

[(5, 1, 1, [0]),
 (5, 1, 5, [0]),
 (5, 1, 9, [0]),
 (5, 1, 13, [0]),
 (5, 1, 17, [0]),
 (5, 1, 21, [0]),
 (5, 1, 25, [0]),
 (5, 1, 29, [0]),
 (5, 1, 33, [0]),
 (5, 1, 37, [0]),
 (5, 1, 41, [0]),
 (10, 1, 1, [0]),
 (10, 1, 5, [0]),
 (10, 1, 9, [0]),
 (10, 1, 13, [0]),
 (10, 1, 17, [0]),
 (10, 1, 21, [0]),
 (10, 1, 25, [0]),
 (10, 1, 29, [0]),
 (10, 1, 33, [0]),
 (10, 1, 37, [0]),
 (10, 1, 41, [0]),
 (10, 1, 45, [0]),
 (10, 1, 49, [0]),
 (10, 1, 53, [0]),
 (10, 1, 57, [0]),
 (10, 1, 61, [0]),
 (10, 1, 65, [0]),
 (10, 1, 69, [0]),
 (10, 1, 73, [0]),
 (10, 1, 77, [0]),
 (10, 1, 81, [0]),
 (10, 1, 85, [0]),
 (10, 1, 89, [0]),
 (10, 1, 93, [0]),
 (10, 1, 97, [0]),
 (10, 1, 101, [0]),
 (10, 1, 105, [0]),
 (10, 1, 109, [0]),
 (10, 1, 113, [0]),
 (10, 1, 117, [0]),
 (10, 1, 121, [0]),
 (10, 1, 125, [0]),
 (10, 1, 129, [0]),
 (10, 1, 133, [0]),
 (10, 1, 137, [0]),
 (10, 1, 141, [0]),
 (10, 1, 145, [0]),
 (10, 1, 149, [0]),
 (10, 1, 153, [0]),
 (11, 1, 1, [0]),
 (11, 1, 5, [0]),
 (11, 1, 9, [0]),

In [15]:
def collate_fn(batch):
    """
    自定义collate函数处理变长序列，生成padding mask
    """
    sequences, targets, lengths = zip(*batch)
    # print(targets)
    
    # 按序列长度排序(降序)
    lengths = torch.tensor(lengths)
    lengths, sort_idx = lengths.sort(descending=True)
    sequences = [sequences[i] for i in sort_idx]
    targets = torch.stack([targets[i] for i in sort_idx])
    
    # 填充序列 (batch_first=True)
    sequences_padded = pad_sequence(sequences, batch_first=True, padding_value=0)
    
    # 创建padding mask (1表示有效数据，0表示padding)
    batch_size, max_len = sequences_padded.shape[0], sequences_padded.shape[1]
    padding_mask = torch.arange(max_len).expand(batch_size, max_len) < lengths.unsqueeze(1)
    padding_mask = padding_mask.float().unsqueeze(-1)  # (batch_size, max_len, 1)
    
    return sequences_padded, targets, padding_mask

In [16]:
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=0, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=0, collate_fn=collate_fn)

In [17]:
len(train_dataset) # 305866 （24-）/ 400904（1-） --stride 8

792447

In [18]:
for x, y, padding_mask in train_loader:
    print(x.shape)
    print(y)
    break

torch.Size([16, 341, 52])
tensor([[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.]])


## 检查设备

In [19]:
# Check device 
# Get the GPU device name if available.
if torch.cuda.is_available():    
    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")
    print('There are %d GPU(s) available. {}'.format(torch.cuda.device_count()))
    print('We will use the GPU: {}'.format(torch.cuda.get_device_name(0)))

# If we dont have GPU but a CPU, training will take place on CPU instead
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")
    
torch.cuda.empty_cache()
    
# Set the seed value all over the place to make this reproducible.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

There are %d GPU(s) available. 1
We will use the GPU: NVIDIA GeForce RTX 2080


## 定义模型

In [20]:
import torch
import torch.nn as nn


class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size=64, num_layers=2, dropout=0.3, num_class=1):
        """
        LSTM模型初始化
        
        参数:
            input_size: 输入特征维度
            hidden_size: 隐藏层大小
            num_layers: LSTM层数
            dropout: Dropout比率
        """
        super(LSTMModel, self).__init__()
        self.num_class = num_class
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0
        )
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(hidden_size, self.num_class)
        
    def forward(self, x, padding_mask):
        # LSTM层
        lstm_out, _ = self.lstm(x)
        
        # # 只取最后一个时间步的输出
        # last_out = lstm_out[:, -1, :]

        # lstm_out = lstm_out * padding_mask
        # 取每个序列最后一个非padding位置的输出
        batch_size = x.size(0)
        lengths = padding_mask.squeeze(-1).sum(dim=1).long()  # 各序列实际长度
        last_out = lstm_out[torch.arange(batch_size), lengths-1, :]  # (batch_size, hidden_size)
        
        # Dropout和全连接层
        out = self.dropout(last_out)
        out = self.fc(out)

        if self.num_class == 1:
            out = torch.sigmoid(out)
        
        return out

In [21]:
# 初始化模型
# model = LSTMModel(input_size=len(feature_cols), hidden_size=256, num_layers=2, dropout=0.1, num_class=1)
model = lstm_itransformerModel(seq_len=24, d_model=64, d_ff=64, e_layers=3, lstm_layers=2, enc_in=len(feature_cols), num_class=2)
# model = lstm_itransformerModel(seq_len=24, d_model=128, d_ff=256, e_layers=3, enc_in=len(feature_cols), lstm_layers=2, num_class=2)

In [22]:
# model.state_dict()

In [23]:
# model.to(device)
# for X, y, padding_mask in train_loader:
#     X, y = X.to(device), y.to(device)
    
#     padding_mask = padding_mask.to(device)
            
#     # 前向传播
#     outputs = model(X,padding_mask)
#     print(X)
#     print(X.shape)
#     print(y.shape)
#     print(outputs)
#     break

In [24]:
# loss = nn.CrossEntropyLoss()(outputs, y.long().squeeze(-1))
# loss

## 训练模型

In [25]:
from tqdm import tqdm
import numpy as np

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

from sklearn.metrics import roc_auc_score, precision_recall_curve, roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.metrics import  average_precision_score

# 训练与评估框架
class ModelTrainer:
    def __init__(self, model, train_loader, val_loader, criterion, optimizer,device='cuda'):
        """
        初始化训练器
        参数:
            model: 模型实例
            train_loader: 训练数据加载器
            val_loader: 验证数据加载器
            device: 训练设备
        """
        self.model = model.to(device)
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.device = device
        self.criterion = criterion
        self.optimizer = optimizer
        self.best_auc = 0
        self.best_model = None
        
    def train_epoch(self, epoch):
        """训练一个epoch"""
        self.model.train()
        total_loss = 0
        progress_bar = tqdm(self.train_loader, desc=f"Epoch {epoch + 1} [Train]")
        
        for X, y, padding_mask in progress_bar:
            X, y = X.to(self.device), y.to(self.device)
            padding_mask = padding_mask.to(self.device)
            
            # 前向传播
            outputs = self.model(X,padding_mask)
            if y.shape[1] == 1:
                loss = self.criterion(outputs, y.long().squeeze(-1))
                # loss = self.criterion(outputs, y)
            else:
                loss = torch.tensor(0.0, device=self.device)
                for i in range(len(outputs)):
                    loss = loss + self.criterion(outputs[i], y[:, i].long().squeeze(-1))
            # # loss = self.criterion(outputs, y)
            # loss = self.criterion(outputs, y.long().squeeze(-1))
            
            # 反向传播和优化
            self.optimizer.zero_grad()
            loss.backward()
            # nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=4.0)
            self.optimizer.step()
            
            total_loss += loss.item() * X.size(0)
            progress_bar.set_postfix(loss=loss.item())
        
        return total_loss / len(self.train_loader.dataset)
    
    def validate(self, testloader):
        """验证模型"""
        self.model.eval()
        val_loss = 0
        all_preds = []
        all_labels = []

        # total_loss = []
        # preds = []
        # trues = []
        
        with torch.no_grad():
            for X, y, padding_mask in tqdm(testloader, desc="Validating"):
                X, y = X.to(self.device), y.to(self.device)
                padding_mask = padding_mask.to(self.device)
                outputs = self.model(X,padding_mask)
                if y.shape[1] == 1:
                    loss = self.criterion(outputs, y.long().squeeze(-1))
                    # loss = self.criterion(outputs, y)
                else:
                    loss = torch.tensor(0.0, device=self.device)
                    for i in range(len(outputs)):
                        loss = loss + self.criterion(outputs[i], y[:, i].long().squeeze(-1))
                # loss = self.criterion(outputs, y)
                
                val_loss += loss.item() * X.size(0)
                all_preds.extend(outputs.cpu().numpy())
                all_labels.extend(y.cpu().numpy())

        
        val_loss /= len(testloader.dataset)
        val_auc = roc_auc_score(all_labels, np.array(all_preds)[:,1])
        val_auprc = average_precision_score(all_labels, np.array(all_preds)[:,1])
        
        # 保存最佳模型
        if val_auc > self.best_auc:
            self.best_auc = val_auc
            self.best_model = self.model.state_dict().copy()
        
        return val_loss, val_auc, val_auprc
    
    def train(self, num_epochs=50, early_stop_patience=5):
        """完整训练流程"""
        train_losses = []
        val_losses = []
        val_aucs = []
        
        no_improve = 0
        for epoch in range(num_epochs):
            train_loss = self.train_epoch(epoch)
            val_loss, val_auc, val_auprc = self.validate(self.val_loader)
            
            train_losses.append(train_loss)
            val_losses.append(val_loss)
            val_aucs.append(val_auc)
            
            print(f"Epoch {epoch + 1}/{num_epochs}:")
            print(f"Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}")
            print(f"Val AUC: {val_auc:.4f} | Val AUPRC: {val_auprc:.4f}")

            torch.save(self.model.state_dict(), f'weights_lstmitransformer_cum_24_2_{epoch}epoch.pth')
            # # 早停机制
            # if val_auc > self.best_auc:
            #     no_improve = 0
            # else:
            #     no_improve += 1
            #     if no_improve >= early_stop_patience:
            #         print(f"Early stopping at epoch {epoch + 1}")
            #         break
        
        # 加载最佳模型
        self.model.load_state_dict(self.best_model)
        return train_losses, val_losses, val_aucs

In [26]:
class FocalLoss(nn.Module):
    def __init__(self, alpha=1, gamma=2, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, inputs, targets):
        BCE_loss = nn.functional.binary_cross_entropy_with_logits(inputs, targets, reduction='none')
        pt = torch.exp(-BCE_loss) # pt = p if target=1, else 1-p
        F_loss = self.alpha * (1-pt)**self.gamma * BCE_loss

        if self.reduction == 'mean':
            return torch.mean(F_loss)
        elif self.reduction == 'sum':
            return torch.sum(F_loss)
        else:
            return F_loss

In [27]:
class MultiClassFocalLoss(nn.Module):
    def __init__(self, alpha=None, gamma=2.0, reduction='mean'):
        super(MultiClassFocalLoss, self).__init__()
        self.alpha = alpha  # 这是一个权重列表，例如 [weight_for_0, weight_for_1]
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, inputs, targets):
        # 先计算标准的交叉熵损失
        ce_loss = nn.functional.cross_entropy(inputs, targets, reduction='none', weight=self.alpha)
        
        # CrossEntropyLoss = LogSoftmax + NLLLoss。
        # 所以我们先对输入做Softmax，得到概率p
        p = torch.exp(-ce_loss) 
        
        # 计算Focal Loss的调制因子 (1 - p_t)^gamma
        # 注意：对于正确类别，p_t就是p。这里ce_loss = -log(p)，所以p = exp(-ce_loss)
        focal_loss = ((1 - p) ** self.gamma) * ce_loss

        if self.reduction == 'mean':
            return focal_loss.mean()
        elif self.reduction == 'sum':
            return focal_loss.sum()
        else:
            return focal_loss

In [28]:
total = len(ys)
num_negative = label_dist[0]
num_positive = label_dist[1]

# 按反比设置，让正类的权重远大于负类
weight_for_1 = num_negative / num_positive
weight_for_0 = 1.0  # 将负类权重设为基准 1.0

# 权重张量的顺序对应类别的索引 [weight_for_class_0, weight_for_class_1]
class_weights = torch.tensor([weight_for_0, weight_for_1], dtype=torch.float).to(device)
class_weights

tensor([ 1.0000, 27.8917], device='cuda:0')

In [29]:
# 训练模型
# criterion = nn.BCEWithLogitsLoss(pos_weight = torch.tensor([0.9783 / (1-0.9783)]).to(device))
# criterion = nn.BCELoss()
criterion = nn.CrossEntropyLoss()
# criterion =  nn.CrossEntropyLoss(weight=class_weights)
# 使用FocalLoss方法
# criterion = MultiClassFocalLoss(alpha=class_weights, gamma=2.0)
# criterion = FocalLoss(alpha=100, gamma=2) # alpha可以设置为正样本权重的近似值

optimizer = optim.Adam(model.parameters(), lr=1e-4)

# 训练模型
trainer = ModelTrainer(model, train_loader, test_loader, criterion, optimizer)
train_losses, val_losses, val_aucs = trainer.train(num_epochs=10)

Validating: 100%|████████████████████████████████████████████████████████████████| 12592/12592 [03:40<00:00, 57.04it/s]


Epoch 1/10:
Train Loss: 0.1147 | Val Loss: 0.1043
Val AUC: 0.8573 | Val AUPRC: 0.4335


Validating: 100%|████████████████████████████████████████████████████████████████| 12592/12592 [03:46<00:00, 55.52it/s]


Epoch 2/10:
Train Loss: 0.1094 | Val Loss: 0.1042
Val AUC: 0.8597 | Val AUPRC: 0.4432


Validating: 100%|████████████████████████████████████████████████████████████████| 12592/12592 [03:36<00:00, 58.14it/s]


...

Epoch 8/10:
Train Loss: 0.0999 | Val Loss: 0.1014
Val AUC: 0.8721 | Val AUPRC: 0.4611


Validating: 100%|████████████████████████████████████████████████████████████████| 12592/12592 [03:37<00:00, 57.94it/s]


Epoch 9/10:
Train Loss: 0.0990 | Val Loss: 0.1026
Val AUC: 0.8713 | Val AUPRC: 0.4526


Validating: 100%|████████████████████████████████████████████████████████████████| 12592/12592 [03:36<00:00, 58.09it/s]


Epoch 10/10:
Train Loss: 0.0974 | Val Loss: 0.1020
Val AUC: 0.8724 | Val AUPRC: 0.4567


In [30]:
# train_loader

In [31]:
# torch.save(model.state_dict(), 'weights_lstmitransformer_cum_24_2_new.pth')

In [32]:
for i in external_database:
    print("Database name: " + i)
    external_data_path = 'icu_mortality_' + i + '.csv'
    df_external = pd.read_csv(file_path + external_data_path)

    external_dataset = TimeSeriesDataset(df_external, feature_cols, mode='cumulative', shuffle=False, stride=4, label=['death_hosp'])
    # external_dataset = TimeSeriesDataset(df_external, feature_cols, window_size=24, forecast_horizon=24, mode='sliding', shuffle=False)
    external_loader = DataLoader(external_dataset, batch_size=16, shuffle=False, num_workers=0, collate_fn=collate_fn)

    total_loss, auroc, auprc = trainer.validate(external_loader)
    print("total_loss, auroc, auprc: ", total_loss, auroc, auprc)

Database name: ams


Validating: 100%|██████████████████████████████████████████████████████████████████| 5414/5414 [01:35<00:00, 56.73it/s]


total_loss, auroc, auprc:  0.10399491340820317 0.8294380628346677 0.3702983660061395
Database name: zhejiang


Validating: 100%|██████████████████████████████████████████████████████████████████| 3717/3717 [01:11<00:00, 52.11it/s]


total_loss, auroc, auprc:  0.10282729920740874 0.8327113291620711 0.3087318947549411
Database name: inspire


...

total_loss, auroc, auprc:  0.10282826358601296 0.6940135496958147 0.11094855608568191
Database name: salz


Validating: 100%|████████████████████████████████████████████████████████████████| 10925/10925 [03:13<00:00, 56.35it/s]


total_loss, auroc, auprc:  0.09038508891366534 0.8033063393721753 0.29497450265693365
Database name: eicu


Validating: 100%|████████████████████████████████████████████████████████████████| 49059/49059 [33:09<00:00, 24.66it/s]


total_loss, auroc, auprc:  0.11559077125284978 0.800404117789405 0.2229475717545797
